# **Comparando velocidade de execução entre código Python e Cython (compilado)**

A ideia aqui é **comparar o tempo de execução** entre código **Python** interpretado e código compilado pelo **Cython / GCC**, usando como função teste um algoritmo de método numérico.

**Cython é um superset / superconjunto de Python**, onde é possível continuar usando a sintaxe do Python porém com mais performance, já que o código é traduzido para linguagem C e então compilado p/ código de máquina.

https://cython.org/

https://cython.readthedocs.io/en/latest/src/userguide/language_basics.html

Python é reconhecida por ser uma linguagem p/ rápido desenvolvimento (maior produtividade do programador), fácil leitura do código e tipagem dinâmica das variáveis (o que facilita muito p/ quem está aprendendo). Porém isso tem um custo: velocidade de execução. Ao indicar os tipos de variáveis p/ o Cython, vamos ganhando em desempenho, porém temos que alterar (muito) o código... o **objetivo** aqui é encontrar um **trade-off entre a a fácil leitura do código e melhor desempenho**, então vamos modificando o código aos poucos e verificamos o tempo de execução de cada implementação.

Para evitar repetição de código aqui no notebook, o código de cada **arquivo .py** é **gerado automaticamente** assim que rodamos todas as células deste notebook, verificar a **aba "Arquivos**" do Colab!

### **Versão do Python e Cython utilizados**

In [1]:
!python3 --version
# !pip install cython --upgrade

import cython, math
from IPython.display import clear_output  #  só para limpar a saída de algumas células

print(f'Cython: {cython.__version__}')

Python 3.10.12
Cython: 3.0.4


### **"snippet" -- Reutilizaremos estes trechos de código várias vezes**

Método da Bissecção (algoritmo de método numérico) que fiz anteriormente em outro notebook,

p/ não ficar repetindo código nas outras células, criamos um código p/ **gerar automaticamente os arquivos .py** que iremos importar mais p/ frente.



In [2]:
procura_raiz_str = '''\
(num, intervalo, precisao):
  """

  :param num: número que iremos extrair a raiz
  :param intervalo: list ou tuple de 2 elementos representando o intervalo min e max, ex: [1, 2]
  :param precisao: precisão da raiz de num, esta função tentará obter erro < precisao
  :return: p (raiz quadrada de num), (erro de n-1, erro), i (passos)
  """

  a, b = intervalo[0], intervalo[1]
  fxa, fxb = fx(a, num), fx(b, num)

  if fxa * fxb < 0:
    p = (a + b) / 2.0
    fxp = fx(p, num)
  elif fxa * fxb > 0:
    return math.nan, (math.nan, math.nan), -1  # retornando número de passos i = -1, intervalo [a b] inválido para achar raiz de num

  i = 0
  while(True):
    i += 1
    if fxp * fxa < 0:
      b, fxb = p, fxp
    elif fxp * fxb < 0:
      a, fxa = p, fxp

    p = (a + b) / 2.0
    erro_anterior = fxp
    fxp = fx(p, num)

    if abs(fxp) < precisao or (p == a) or (p == b):  # abs(erro) < precisao; com precisao < 5 * 10^(-16) ficamos preso dentro do loop, ver sys.float_info
      break

  return p, (erro_anterior, fxp), i
'''  # fim do procura_raiz_str


In [3]:
procura_raiz_str_b = '''\
(num: float,
                         intervalo: list[float, float] | tuple[float, float],
                         precisao: float
                         ) -> tuple[float, tuple[float, float], int]:
                         # Usando PEP 484 (Type Hints / Type Annotations of function parameters)
                         # e PEP 526 (Variable Annotations)
  """

  :param num: número que iremos extrair a raiz
  :param intervalo: list ou tuple de 2 elementos representando o intervalo min e max, ex: [1, 2]
  :param precisao: precisão da raiz de num, esta função tentará obter erro < precisao
  :return: p (raiz quadrada de num), (erro de n-1, erro), i (passos)
  """

  a : float = intervalo[0]
  b : float = intervalo[1]
  fxa, fxb = fx(a, num), fx(b, num)

  # p : float = math.nan
  # fxp : float = math.nan
  # erro_anterior : float = math.nan

  if fxa * fxb < 0:
    p = (a + b) / 2.0
    fxp = fx(p, num)
  elif fxa * fxb > 0:
    return math.nan, (math.nan, math.nan), -1  # retornando número de passos i = -1, intervalo [a b] inválido para achar raiz de num

  i : int = 0

  while(True):
    i += 1
    if fxp * fxa < 0:
      b, fxb = p, fxp
    elif fxp * fxb < 0:
      a, fxa = p, fxp

    p = (a + b) / 2.0
    erro_anterior = fxp
    fxp = fx(p, num)

    if abs(fxp) < precisao or (p == a) or (p == b):  # abs(erro) < precisao; com precisao < 5 * 10^(-16) ficamos preso dentro do loop, ver sys.float_info
      break

  return p, (erro_anterior, fxp), i
'''  # fim do procura_raiz_str_b, usando PEP 484 (Type Hints) e PEP 526 (Variable Annotations)


In [4]:

file_base_name = 'bisseccao'  # nome base dos arquivos *.py que iremos gerar

# abaixo funções para criar automaticamente os arquivos *.py / setup.py

def func_header(sufix_str):
  return f'def procura_raiz{sufix_str}'

def setup_text(file_to_build):
  setup_py_content = ('from setuptools import setup\n'
                      'from Cython.Build import cythonize\n'
                      '\n'
                      'setup(\n'
                      f'    ext_modules = cythonize("{file_to_build}")\n'
                      ')\n')
  return setup_py_content

def create_my_py_file(text_str, base_name_str, sufix_str='', file_extension='py'):
  with open(f'{base_name_str}{sufix_str}.{file_extension}', mode='w', encoding='utf-8') as file:
    file.write(text_str)

def build_py_file(fx_str, sufix_str, base_code_str=procura_raiz_str, create_setup=True):

  create_my_py_file(fx_str + func_header(sufix_str) + base_code_str,
                     file_base_name,
                     sufix_str)  # para criar o nosso *.py

  if create_setup:
    create_my_py_file(setup_text(file_base_name + sufix_str + '.py'), 'setup')  # aqui criamos o setup.py p/ poder compilar

    !python setup.py build_ext --inplace  # compilando o código, aqui vai criar um arquivo .so que importaremos depois

    clear_output()  # só p/ limpar o output desta célula


### **Implementação 1 - Python Interpreter puro (sem compilar):**

Abrir o "**bisseccao_imp_1.py**" que irá gerar, **ver na aba Arquivos do Colab**. Abaixo um pequeno trecho do código gerado, nas próximas células iremos alterar essa função fx().


In [32]:
fx_str_1 = \
"""
import math

def fx(x, num):
  return x ** 2 - num

"""

build_py_file(fx_str_1, '_impl_1', create_setup=False)  # criando o arquivo .py, ver na aba 'Arquivos' do Colab


### **Implementação 2 - compilando com o Cython, porém sem modificações no código**

Estamos usando o mesmo código da implementação 1.

Para compilar o código, estamos usando o **setuptools** e um arquivo **setup.py** (que é gerado ao rodar a célula seguinte).

https://cython.readthedocs.io/en/latest/src/quickstart/build.html#building-a-cython-module-using-setuptools

In [6]:
build_py_file(fx_str_1, '_impl_2')  # estaremos usando o mesmo código da implementação 1, fx_str_1 da célula anterior


### **Implementação 3 - type hint na fx()**

Começamos a alterar o código a partir da função **fx()** já que a mesma é chamada várias vezes por conta do loop em **procura_raiz_impl_X()** e o programa vai gastar boa parte do tempo lá.

Vamos indicar os tipos das variáveis p/ o Cython com **type hint / PEP 484**


In [37]:
fx_str = \
"""
import math

def fx(x: float, num: float) -> float:  # usando PEP 484 / Type Hints
  return x ** 2 - num

"""

build_py_file(fx_str, '_impl_3')  # criando o arquivo .py e o setup.py, e compilando (ver a célula "snippet"), vai ser gerado um .so que iremos importar depois


### **Implementação 3B - type hint na fx() + variable annotations**

Além de **type hint na fx()**, vamos usar **variable annotations / PEP 526** na função **procura_raiz_impl_XX()**.

Ver como ficou o código na variável **procura_raiz_str_b**, na célula **snippet** no começo deste notebook, ou no **.py gerado na aba "Arquivos" do Colab**.

In [38]:
fx_str = \
"""
import cython, math

def fx(x: float, num: float) -> float:  # usando PEP 484 / Type Hints
  return x ** 2 - num

"""

build_py_file(fx_str, '_impl_3B', base_code_str=procura_raiz_str_b)  # usaremos o código do procura_raiz_str_b que contém variable annotations (PEP 526)


### **Implementação 4 - type hint na fx() + decorador @cython.cfunc**

Vamos usar o **decorador @cython.cfunc**, que vai transformar a fx() em uma **função de linguagem C** (não poderemos chamar ela fora do módulo que compilamos, em contrapartida ganhamos em velocidade de execução). Além disso mantemos o **type hint / PEP 484** na fx(). No restante do código voltamos p/ o original, sem alterações.

Precisamos **importar a biblioteca cython** p/ usar o decorador!


In [9]:
fx_str = \
"""
import cython, math

@cython.cfunc
def fx(x: float, num: float) -> float:  # usando PEP 484 / Type Hints
  return x ** 2 - num

"""

build_py_file(fx_str, '_impl_4')


### **Implementação 4B - type hint na fx() + decorador @cython.cfunc + variable annotations**

Além de type hint / PEP 484 e decorador @cython.cfunc na fx(), vamos usar **variable annotations / PEP (526)**, ou seja, vamos **especificar os tipos de cada variável** dentro da **procura_raiz_impl_XX()**.

In [10]:
fx_str = \
"""
import cython, math

@cython.cfunc
def fx(x: float, num: float) -> float:  # usando PEP 484 / Type Hints
  return x ** 2 - num

"""

build_py_file(fx_str, '_impl_4B', base_code_str=procura_raiz_str_b)  # usaremos o código do procura_raiz_str_b que contém variable annotations (PEP 526)


### **Implementação 5 - sintaxe 'Pure Python' do Cython**

Aqui importaremos a biblioteca Cython e usaremos os tipos disponíveis como **cython.double** e **cython.int**, além de usar o decorador **@cython.cfunc**. Este código ainda é **compatível com interpretador Python**, daí o nome **Pure Python**. Podemos rodar como se fosse um script Python, e também podemos compilar o código usando Cython + GCC ou outro compilador.

Aqui vamos compilar o código direto na célula do notebook (comando mágico ***%%cython***):

https://cython.readthedocs.io/en/latest/src/quickstart/build.html#using-the-jupyter-notebook

In [11]:
# Isso aqui carrega a extensão Cython aqui no Jupyter / Colab
%load_ext cython

In [ ]:
%%cython
# cython: language_level=3

# comentário acima é um header especial, compiler directives

import cython, math

@cython.cfunc
def fx(x: cython.double, num: cython.double) -> cython.double:
  return x ** 2 - num

def procura_raiz_impl_5(num: cython.double,
                        intervalo: tuple[cython.double, cython.double],
                        precisao: cython.double
                        ) -> tuple[cython.double, tuple[cython.double, cython.double], cython.int]:
  """

  :param num: número que iremos extrair a raiz
  :param intervalo: list ou tuple de 2 elementos representando o intervalo min e max, ex: [1, 2]
  :param precisao: precisão da raiz de num, esta função tentará obter erro < precisao
  :return: p (raiz quadrada de num), (erro de n-1, erro), i (passos)
  """

  a: cython.double = intervalo[0]
  b: cython.double = intervalo[1]
  fxa: cython.double = fx(a, num)
  fxb: cython.double = fx(b, num)

  # p: cython.double = math.nan  # só para não receber warning na compilação
  # fxp: cython.double = math.nan
  # erro_anterior: cython.double = math.nan

  if fxa * fxb < 0:
    p = (a + b) / 2.0
    fxp = fx(p, num)
  elif fxa * fxb > 0:
    return math.nan, (math.nan, math.nan), -1  # retornando número de passos i = -1, intervalo [a b] inválido para achar raiz de num

  i: cython.int = 0
  while(True):
    i += 1
    if fxp * fxa < 0:
      b, fxb = p, fxp
    elif fxp * fxb < 0:
      a, fxa = p, fxp

    p = (a + b) / 2.0
    erro_anterior = fxp
    fxp = fx(p, num)

    if abs(fxp) < precisao or (p == a) or (p == b):  # abs(erro) < precisao; com precisao < 5 * 10^(-16) ficamos preso dentro do loop, ver sys.float_info
      break

  return p, (erro_anterior, fxp), i


### **Implementação 5B - decoradores @cython.cfunc + @cython.locals**

Na célula anterior, poderíamos simplesmente ter usado o **decorador @cython.locals()** para definir os tipos dos argumentos e variáveis, ao invés de modificar o código. Estamos utilizando também o decorador **@cython.returns()** p/ indicar o tipo que as funções retornam.

Ver na aba "Arquivos" do Colab como ficou o código no **arquivo .py gerado**.

In [13]:
fx_str = \
"""
import cython, math

@cython.cfunc
@cython.returns(cython.double)
@cython.locals(x=cython.double, num=cython.double)
def fx(x, num):
  return x ** 2 - num

@cython.locals(num=cython.double,
               intervalo=tuple[cython.double, cython.double],
               precisao=cython.double,
               a=cython.double,
               b=cython.double,
               fxa=cython.double,
               fxb=cython.double,
               p=cython.double,
               fxp=cython.double,
               erro_anterior=cython.double,
               i=cython.int)
@cython.returns(tuple[cython.double, tuple[cython.double, cython.double], cython.int])
"""

build_py_file(fx_str, '_impl_5B')  # usaremos o código do procura_raiz_str original, sem modificações, os tipos dos parâmetros e variáveis estão indicados no decorador @cython.locals


### **Implementação 6 - usando sintaxe específica Cython: cdef**

Aqui o código já começou a ficar um pouco longe do Python e **mais próximo de C/C++**, usamos a **sintaxe específica do Cython cdef**. Se fossemos salvar um arquivo com o código abaixo, teríamos que usar a extensão **.pyx**

Este código **não é compatível com o interpretador Python!** Foi feito para ser compilado.

In [ ]:
%%cython
# cython: language_level=3, warn.maybe_uninitialized=False

#import math
from cython.cimports.libc.math import NAN

cdef double fx(double x, double num):  # cdef faz as funções virarem 'C functions', não é mais visivel no código Python (não podemos chamar diretamente)
  return x ** 2 - num

def procura_raiz_impl_6(double num,
                        (double, double) intervalo,
                        double precisao
                        ) -> tuple[double, tuple[double, double], int]:
  """

  :param num: número que iremos extrair a raiz
  :param intervalo: list ou tuple de 2 elementos representando o intervalo min e max, ex: [1, 2]
  :param precisao: precisão da raiz de num, esta função tentará obter erro < precisao
  :return: p (raiz quadrada de num), (erro de n-1, erro), i (passos)
  """

  cdef double a = intervalo[0]
  cdef double b = intervalo[1]
  cdef double fxa = fx(a, num)
  cdef double fxb = fx(b, num)

  # cdef double p = NAN
  # cdef double fxp = NAN
  # cdef double erro_anterior = NAN

  if fxa * fxb < 0:
    p = (a + b) / 2.0
    fxp = fx(p, num)
  elif fxa * fxb > 0:
    return NAN, (NAN, NAN), -1  # retornando número de passos i = -1, intervalo [a b] inválido para achar raiz de num

  cdef int i = 0
  while(True):
    i += 1
    if fxp * fxa < 0:
      b, fxb = p, fxp
    elif fxp * fxb < 0:
      a, fxa = p, fxp

    p = (a + b) / 2.0
    erro_anterior = fxp
    fxp = fx(p, num)

    if abs(fxp) < precisao or (p == a) or (p == b):  # abs(erro) < precisao; com precisao < 5 * 10^(-16) ficamos preso dentro do loop, ver sys.float_info
      break

  return p, (erro_anterior, fxp), i


### **Implementação 7 - usando um arquivo .pxd p/ compilar**

Ao invés de alterarmos o código python no arquivo .py, podemos criar um **arquivo .pxd (augmenting .pxd)**, as declarações de funções e tipos das variáveis são sobrescritos neste arquivo.

https://cython.readthedocs.io/en/latest/src/tutorial/pure.html#augmenting-pxd

Em outras palavras: o projeto / código fica em **2 arquivo indepentendentes**, um totalmente compatível com **Python (com a tipagem dinâmica e sem alterações)** e outro p/ o **Cython (com tipagem estática, sobrescrevendo o outro arquivo)**.

In [33]:
pxd_str = \
"""
# cython: language_level=3

import cython, math

cdef double fx(double x, double num)  # podemos usar a sintaxe específica do Cython

@cython.locals(a=cython.double,
               b=cython.double,
               fxa=cython.double,
               fxb=cython.double,
               p=cython.double,
               fxp=cython.double,
               erro_anterior=cython.double,
               i=cython.int)
cpdef (double, (double, double), int) procura_raiz_impl_7(double num,
                                                          (double, double) intervalo,
                                                          double precisao)
                                                          # podemos também usar type hints / PEP 484,
                                                          # e também misturar com sintaxe específica do Cython
"""

create_my_py_file(pxd_str, file_base_name, '_impl_7', file_extension='pxd')  # criando o arquivo .pxd

build_py_file(fx_str_1, '_impl_7')  # código será o mesmo da implementação 1, compilando o código, usando os arquivos bisseccao_impl_7.py e bisseccao_impl_7.pxd


### **Visualizando os arquivos criados até agora / importando as bibliotecas**

In [16]:
# listando os arquivos no nosso diretório / pasta, vamos ver todos os arquivos gerados até aqui, de olho nos *.so compilados, estamos chamando as funções de lá
!ls

bisseccao_impl_1.py
bisseccao_impl_2.c
bisseccao_impl_2.cpython-310-x86_64-linux-gnu.so
bisseccao_impl_2.py
bisseccao_impl_3B.c
bisseccao_impl_3B.cpython-310-x86_64-linux-gnu.so
bisseccao_impl_3B.py
bisseccao_impl_3.c
bisseccao_impl_3.cpython-310-x86_64-linux-gnu.so
bisseccao_impl_3.py
bisseccao_impl_4B.c
bisseccao_impl_4B.cpython-310-x86_64-linux-gnu.so
bisseccao_impl_4B.py
bisseccao_impl_4.c
bisseccao_impl_4.cpython-310-x86_64-linux-gnu.so
bisseccao_impl_4.py
bisseccao_impl_5B.c
bisseccao_impl_5B.cpython-310-x86_64-linux-gnu.so
bisseccao_impl_5B.py
bisseccao_impl_7.c
bisseccao_impl_7.cpython-310-x86_64-linux-gnu.so
bisseccao_impl_7.pxd
bisseccao_impl_7.py
build
sample_data
setup.py


In [17]:
# Vamos carregar aqui todas as implementações geradas / compiladas:

from bisseccao_impl_1 import procura_raiz_impl_1    # Python Interpreter, NÃO compilado
from bisseccao_impl_2 import procura_raiz_impl_2    # código compilado com Cython / GCC (sem modificações no código)
from bisseccao_impl_3 import procura_raiz_impl_3    # type hint / PEP 484 na fx()
from bisseccao_impl_3B import procura_raiz_impl_3B  # type hint / PEP 484 na fx() + variable annotations / PEP 526 no restante do código
from bisseccao_impl_4 import procura_raiz_impl_4    # type hint / PEP 484 na fx() + decorador @cython.cfunc
from bisseccao_impl_4B import procura_raiz_impl_4B  # type hint / PEP 484 na fx() + decorador @cython.cfunc + variable annotations / PEP 526 no restante do código
from bisseccao_impl_5B import procura_raiz_impl_5B  # decoradores @cython.cfunc + @cython.locals
from bisseccao_impl_7 import procura_raiz_impl_7    # usando um arquivo .pxd p/ compilar

# as funções procura_raiz_impl_5() e procura_raiz_impl_6() já estão definidos pelas células anteriores!

### **Definição dos parâmetros de entrada p/ os testes**

In [18]:
# ENTRADA
num = 2      # num é o número que se quer achar a raiz
intervalo_ab = [1, 2]  # intervalo

# precisao = 0.0000000000000005  # limite é 5 * 10^(-16)
precisao   = 0.000000000000001

In [19]:
# EXTRAINDO A RAIZ, chamando a função
x, erros, n = procura_raiz_impl_1(num, intervalo_ab, precisao)

# SAÍDA, mostrando na tela
if n >= 0:
  print(f'\nIremos extrair a raiz quadrada de: {num}\nIntervalo de busca: {intervalo_ab}')
  print(f'\nerro(n-1) = {abs(erros[0]):.16f}\nerro =      {abs(erros[1]):.16f}\nprecisao =  {precisao:.15f}\n\nraiz de {num} (aprox.):      {x} --> x^2 = {x ** 2}\nraiz de {num} (math.sqrt()): {math.sqrt(num)}')
  print(f'\nnumero de passos: {n}')
if n == -1:
  print(f'\nos números do intervalo {intervalo_ab} são inválidos p/ se obter a raiz de {num}, forneça outro intervalo.')
if precisao < abs(erros[1]):
  print(f'\nraiz obtida tem precisao de {erros[1]} (valor do erro), o valor de precisão escolhido é muito baixo ({precisao}), escolha um número maior p/ a precisão.')


Iremos extrair a raiz quadrada de: 2
Intervalo de busca: [1, 2]

erro(n-1) = 0.0000000000000029
erro =      0.0000000000000004
precisao =  0.000000000000001

raiz de 2 (aprox.):      1.414213562373095 --> x^2 = 1.9999999999999996
raiz de 2 (math.sqrt()): 1.4142135623730951

numero de passos: 49


### **Comparação de tempo (%timeit)**

Temos um ganho de desempenho aproximadamente de **37x** entre um script comum que roda em Python Interpreter (sem anotações de tipo das variáveis) e código usando sintaxe específica do Cython.

In [30]:
# 1 - Interpretador Python puro
%timeit procura_raiz_impl_1(num, intervalo_ab, precisao)

16.5 µs ± 2.26 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [21]:
# 2 - Compilando o código com Cython / GCC (sem modificações no código)
%timeit procura_raiz_impl_2(num, intervalo_ab, precisao)

8.87 µs ± 50.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [22]:
# 3 - type hint / PEP 484 na fx()
%timeit procura_raiz_impl_3(num, intervalo_ab, precisao)

7.81 µs ± 1.8 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [23]:
# 3B - type hint / PEP 484 na fx() + variable annotations / PEP 526 no restante do código
%timeit procura_raiz_impl_3B(num, intervalo_ab, precisao)

5.05 µs ± 42.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [24]:
# 4 - # type hint / PEP 484 na fx() + decorador @cython.cfunc
%timeit procura_raiz_impl_4(num, intervalo_ab, precisao)

3.8 µs ± 97.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [31]:
# 4B - type hint / PEP 484 na fx() + decorador @cython.cfunc + variable annotations / PEP 526 no restante do código
%timeit procura_raiz_impl_4B(num, intervalo_ab, precisao)

669 ns ± 5.74 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [26]:
# 5 - sintaxe 'Pure Python' do Cython
%timeit procura_raiz_impl_5(num, intervalo_ab, precisao)

415 ns ± 3.58 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [27]:
# 5B - decoradores @cython.cfunc + @cython.locals
%timeit procura_raiz_impl_5B(num, intervalo_ab, precisao)

439 ns ± 45 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [28]:
# 6 - sintaxe específica do Cython: cdef
%timeit procura_raiz_impl_6(num, intervalo_ab, precisao)

438 ns ± 44.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [29]:
# 7 - Utilizando um arquivo .pxd, cdef na fx() + @cython.locals
%timeit procura_raiz_impl_7(num, intervalo_ab, precisao)

372 ns ± 5.11 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### **Conclusão**

Só de ter compilado o código sem alterações ganhamos em desempenho (**aprox. 2x**), conforme vamos indicando os tipos das variáveis com recursos do próprio Python (*type hint* / PEP 484 e *variable annotations* / PEP 526) há melhora de performance, conforme pode ser visto entre as implementações 3 a 4B, porém tivemos que alterar o código. Usar o decorador @cython.cfunc ajudou também no desempenho (chamada de funções C são mais rápidas que funções Python).

Nas implementações 5 e 6 alteramos bastante o código, porém foi onde tivemos melhor performance (**37x !!!**). Se fosse escolher entre um dos dois, ficaria com o "Pure Python" do Cython, em 5.

Em 5B e no 7, não precisamos alterar tanto o código de origem, usando apenas os decoradores @cython.cfunc e @cython.locals ou o arquivo à parte (.pxd do 7), e ganhamos desempenho comparável às implementações 5 e 6. **Entre todas opções, ficaria com 5B e 7, já que foi mais fácil implementar e mexemos menos no código original** (o que faz que corramos menos risco de introduzir algum bug ou fazer com que o código fique acidentalmente incompatível com o interpretador Python).

Abaixo algumas leituras que achei interessante à respeito do **Cython**.

[What is Cython? Python at the speed of C](https://www.infoworld.com/article/3250299/what-is-cython-python-at-the-speed-of-c.html)

[An Introduction to Just Enough Cython to be Useful](https://www.peterbaumgartner.com/blog/intro-to-just-enough-cython-to-be-useful/)

[Some reasons to avoid Cython](https://pythonspeed.com/articles/cython-limitations/)